*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  ###!pip install tf-nightly
  print("Aleardy done !")
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
###!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [29]:
# get data files
#!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
#!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = f".\..\data\\fcc_sms_text_classification\train-data.tsv"
test_file_path = f".\..\data\\fcc_sms_text_classification\valid-data.tsv"

In [81]:
train_data= pd.read_csv("../data/fcc_sms_text_classification/train-data.tsv", on_bad_lines='skip',sep=";;;",engine='python')
test_data=pd.read_csv("../data/fcc_sms_text_classification/valid-data.tsv", on_bad_lines='skip',sep=";;;",engine='python')

In [82]:
train_data.head()

,sms_class,message
0,ham,ahhhh...just woken up!had a bad dream about u...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like!..."
3,ham,mum say we wan to go then go... then she can ...
4,ham,never y lei... i v lazy... got wat? dat day ü...


In [84]:
df_train=train_data[["message","sms_class"]]
df_test=test_data[["message","sms_class"]]

In [85]:
df_train

,message,sms_class
0,ahhhh...just woken up!had a bad dream about u...,ham
1,you can never do nothing,ham
2,"now u sound like manky scouse boy steve,like!...",ham
3,mum say we wan to go then go... then she can ...,ham
4,never y lei... i v lazy... got wat? dat day ü...,ham
...,...,...
4153,just woke up. yeesh its late. but i didn't fa...,ham
4154,what do u reckon as need 2 arrange transport ...,ham
4155,free entry into our £250 weekly competition j...,spam
4156,-pls stop bootydelious (32/f) is inviting you...,spam


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):



  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
